📒 [NB 2-3] 범주형 데이터 인코딩 (Encoding)
1. 데이터 준비
컴퓨터는 '문자'를 이해하지 못한다. 머신러닝 분석을 위해 문자로 된 범주형 데이터를 숫자로 변환하는 과정을 실습한다.

In [2]:
!pip install pandas scikit-learn

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 가상 환자 데이터
data = {
    'Patient_ID': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'Gender': ['Male', 'Female', 'Female', 'Male', 'Male'],
    'Blood_Type': ['A', 'B', 'O', 'AB', 'A'],
    'Severity': ['Mild', 'Severe', 'Moderate', 'Mild', 'Severe'] # 중증도 (순서 있음)
}

df = pd.DataFrame(data)
print("원본 데이터:")
display(df)

원본 데이터:


,Patient_ID,Gender,Blood_Type,Severity
0,P1,Male,A,Mild
1,P2,Female,B,Severe
2,P3,Female,O,Moderate
3,P4,Male,AB,Mild
4,P5,Male,A,Severe


2. Label Encoding (순서가 있는 경우)
중증도(Severity)처럼 Mild < Moderate < Severe 순서가 있는 경우, 숫자의 크기로 정보를 표현할 수 있다.

In [3]:
# 1. 맵핑 딕셔너리 직접 정의 (가장 안전하고 확실한 방법)
severity_mapping = {'Mild': 0, 'Moderate': 1, 'Severe': 2}

# 2. map 함수 적용
df['Severity_Encoded'] = df['Severity'].map(severity_mapping)

print("Label Encoding 결과:")
display(df[['Patient_ID', 'Severity', 'Severity_Encoded']])

Label Encoding 결과:


,Patient_ID,Severity,Severity_Encoded
0,P1,Mild,0
1,P2,Severe,2
2,P3,Moderate,1
3,P4,Mild,0
4,P5,Severe,2


3. One-Hot Encoding (순서가 없는 경우)
혈액형(A, B, O, AB)에는 순서가 없다. A형(0) < B형(1)이라고 표현하면 모델이 오해할 수 있다. 이럴 때는 각 범주를 별도의 컬럼(0/1)으로 만든다.

In [4]:
# Pandas의 get_dummies 함수 사용 (가장 간편함)
# prefix: 생성될 컬럼 이름 앞에 붙일 접두사
blood_dummies = pd.get_dummies(df['Blood_Type'], prefix='Blood')

print("One-Hot Encoding 결과 (가변수 생성):")
display(blood_dummies)

# 원본 데이터와 합치기
df_final = pd.concat([df, blood_dummies], axis=1)
display(df_final)

One-Hot Encoding 결과 (가변수 생성):


,Blood_A,Blood_AB,Blood_B,Blood_O
0,True,False,False,False
1,False,False,True,False
2,False,False,False,True
3,False,True,False,False
4,True,False,False,False


,Patient_ID,Gender,Blood_Type,Severity,Severity_Encoded,Blood_A,Blood_AB,Blood_B,Blood_O
0,P1,Male,A,Mild,0,True,False,False,False
1,P2,Female,B,Severe,2,False,False,True,False
2,P3,Female,O,Moderate,1,False,False,False,True
3,P4,Male,AB,Mild,0,False,True,False,False
4,P5,Male,A,Severe,2,True,False,False,False


4. Scikit-learn을 이용한 인코딩 (참고)
머신러닝 파이프라인 구축 시에는 Scikit-learn의 인코더를 주로 사용한다.

In [5]:
# LabelEncoder 사용 예시 (Gender: Male/Female)
le = LabelEncoder()
df['Gender_Code'] = le.fit_transform(df['Gender'])

print("Scikit-learn LabelEncoder 결과:")
# 0과 1이 각각 무엇을 의미하는지 확인
print(f"Classes: {le.classes_}") # ['Female', 'Male'] -> Female=0, Male=1 (알파벳 순)
display(df[['Gender', 'Gender_Code']])

Scikit-learn LabelEncoder 결과:
Classes: ['Female' 'Male']


,Gender,Gender_Code
0,Male,1
1,Female,0
2,Female,0
3,Male,1
4,Male,1
